!pip install dateparser

In [1]:
import random
import json
import csv
import numpy as np
import pandas as pd
import datetime
import dateparser

import requests
from requests.auth import HTTPBasicAuth


%load_ext dotenv
#%reload_ext dotenv

# Use find_dotenv to locate the file
%dotenv
import os

In [2]:
HOST = "http://localhost:5000"

# auto daily report (flex query)

In [ ]:
ib = {
    'QUERY_ID': '332646',
    'TOKEN': os.getenv('TOKEN')
}


res = requests.post(
    f'{HOST}/reports/ib/eod/v2', 
    json=ib, 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')))
json.loads(res.content)

In [ ]:
ib = {
    'QUERY_ID': '332646',
    'TOKEN': os.getenv('TOKEN')
}

res = requests.get(
    f'{HOST}/reports/ib/eod/v2/reportDate', 
    json=ib, 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')))
json.loads(res.content)

# xls fa_data

In [ ]:
res = requests.get(
    f'{HOST}/reports/ib/eod/v2', 
#    json=json.loads(requests.get('http://127.0.0.1:443/executions').content),
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')))
json.loads(res.content)

In [4]:
json.loads(requests.get('http://127.0.0.1:443/executions').content)

{'execDetails': [{'execution': {'m_acctNumber': 'F2571253',
    'm_orderId': -3403,
    'm_evRule': None,
    'm_time': '20190308  02:15:00',
    'm_permId': 1363089126,
    'm_evMultiplier': 0,
    'm_liquidation': 0,
    'm_orderRef': None,
    'm_price': 11319.0,
    'm_avgPrice': 11319.0,
    'm_cumQty': 1,
    'm_side': 'BOT',
    'm_clientId': 0,
    'm_shares': 1,
    'm_execId': '00016952.5c81a999.01.01',
    'm_exchange': 'HKFE'},
   'contract': {'m_tradingClass': 'HHI',
    'm_symbol': 'HHI.HK',
    'm_conId': 327970755,
    'm_secType': 'FUT',
    'm_includeExpired': False,
    'm_right': None,
    'm_multiplier': '50',
    'm_expiry': '20190328',
    'm_currency': 'HKD',
    'm_localSymbol': 'HHIH9',
    'm_exchange': 'HKFE',
    'm_strike': 0.0}},
  {'execution': {'m_acctNumber': 'U2621631',
    'm_orderId': -3403,
    'm_evRule': None,
    'm_time': '20190308  02:15:00',
    'm_permId': 1105590061,
    'm_evMultiplier': 0,
    'm_liquidation': 0,
    'm_orderRef': None,
 

In [51]:
tws_executions = json.loads(requests.get('http://127.0.0.1:443/executions').content)
len(tws_executions['execDetails'])

122

In [52]:
data = []
for exec in tws_executions['execDetails']:
    
    # flatten dict
    f_exec = {}
    for key in exec:
        for key_data in exec[key]:
            f_exec[f'{key}_{key_data}'] = exec[key][key_data]
    
    data.append(f_exec)
    
    #break
    
execs = pd.DataFrame(data)
execs

,contract_m_conId,contract_m_currency,contract_m_exchange,contract_m_expiry,contract_m_includeExpired,contract_m_localSymbol,contract_m_multiplier,contract_m_right,contract_m_secType,contract_m_strike,...,execution_m_exchange,execution_m_execId,execution_m_liquidation,execution_m_orderId,execution_m_orderRef,execution_m_permId,execution_m_price,execution_m_shares,execution_m_side,execution_m_time
0,327970755,HKD,HKFE,20190328,False,HHIH9,50,None,FUT,0.0,...,HKFE,00016952.5c81a999.01.01,0,-3403,None,1363089126,11319.00000,1,BOT,20190308 02:15:00
1,327970755,HKD,HKFE,20190328,False,HHIH9,50,None,FUT,0.0,...,HKFE,U+00014c06.5c80cbc2.01.01,0,-3403,None,1105590061,11319.00000,1,BOT,20190308 02:15:00
2,299552802,USD,GLOBEX,20190315,False,ESH9,50,None,FUT,0.0,...,GLOBEX,0000d8df.5c81a4a6.01.01,0,-3405,None,1808854251,2746.00000,1,SLD,20190308 04:52:54
3,299552802,USD,GLOBEX,20190315,False,ESH9,50,None,FUT,0.0,...,GLOBEX,U+00014c06.5c80cbc6.01.01,0,-3405,None,1105590055,2746.00000,1,SLD,20190308 04:52:54
4,327970755,HKD,HKFE,20190328,False,HHIH9,50,None,FUT,0.0,...,HKFE,00016952.5c81ce66.01.01,0,-3402,None,1363089127,11226.00000,1,SLD,20190308 04:56:42
5,327970755,HKD,HKFE,20190328,False,HHIH9,50,None,FUT,0.0,...,HKFE,U+00014c06.5c80cbca.01.01,0,-3402,None,1105590062,11226.00000,1,SLD,20190308 04:56:42
6,299552989,USD,GLOBEX,20190315,False,NQH9,20,None,FUT,0.0,...,GLOBEX,0000d94c.5c81a1ea.01.01,0,2147483647,None,2120345848,7017.75000,1,SLD,20190308 05:02:46
7,13906061,JPY,TSEJ,None,False,9202,None,None,STK,0.0,...,TSEJ,00015232.5c81abe6.01.01,0,2147483647,None,2120345856,3994.00000,100,SLD,20190308 05:03:23
8,299552802,USD,GLOBEX,20190315,False,ESH9,50,None,FUT,0.0,...,GLOBEX,0000d8df.5c81a4b8.01.01,0,2147483647,None,2120345866,2744.75000,1,SLD,20190308 05:04:17
9,299552802,USD,GLOBEX,20190315,False,ESH9,50,None,FUT,0.0,...,GLOBEX,0000d8df.5c81a4b9.01.01,0,2147483647,None,2120345871,2745.00000,1,SLD,20190308 05:04:44


In [53]:
def add_sign_qty(row):
    return row['execution_m_shares'] if row['execution_m_side'] == 'BOT' else -row['execution_m_shares'] 

In [54]:
execs['execution_m_shares'] = execs.apply(add_sign_qty, axis=1)

execs.head()

,contract_m_conId,contract_m_currency,contract_m_exchange,contract_m_expiry,contract_m_includeExpired,contract_m_localSymbol,contract_m_multiplier,contract_m_right,contract_m_secType,contract_m_strike,...,execution_m_exchange,execution_m_execId,execution_m_liquidation,execution_m_orderId,execution_m_orderRef,execution_m_permId,execution_m_price,execution_m_shares,execution_m_side,execution_m_time
0,327970755,HKD,HKFE,20190328,False,HHIH9,50,None,FUT,0.0,...,HKFE,00016952.5c81a999.01.01,0,-3403,None,1363089126,11319.0,1,BOT,20190308 02:15:00
1,327970755,HKD,HKFE,20190328,False,HHIH9,50,None,FUT,0.0,...,HKFE,U+00014c06.5c80cbc2.01.01,0,-3403,None,1105590061,11319.0,1,BOT,20190308 02:15:00
2,299552802,USD,GLOBEX,20190315,False,ESH9,50,None,FUT,0.0,...,GLOBEX,0000d8df.5c81a4a6.01.01,0,-3405,None,1808854251,2746.0,-1,SLD,20190308 04:52:54
3,299552802,USD,GLOBEX,20190315,False,ESH9,50,None,FUT,0.0,...,GLOBEX,U+00014c06.5c80cbc6.01.01,0,-3405,None,1105590055,2746.0,-1,SLD,20190308 04:52:54
4,327970755,HKD,HKFE,20190328,False,HHIH9,50,None,FUT,0.0,...,HKFE,00016952.5c81ce66.01.01,0,-3402,None,1363089127,11226.0,-1,SLD,20190308 04:56:42


In [55]:
wm = lambda x: np.average(x, weights=execs.loc[x.index, "execution_m_shares"])

In [56]:
f = {'execution_m_shares': ['sum'], 'execution_m_price': {'weighted_mean' : wm} }

In [57]:
execs.groupby(["contract_m_localSymbol", "execution_m_side"]).agg(f)

c:\users\gregory\appdata\local\conda\conda\envs\notebook\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,,execution_m_shares,execution_m_price
,,sum,weighted_mean
contract_m_localSymbol,execution_m_side,,
1109,SLD,-10000,30.250000
291,SLD,-2000,30.050000
6030,SLD,-7500,18.337333
9202,SLD,-100,3994.000000
ADYEN,SLD,-10,634.400000
AMZN,BOT,60,1615.816667
BALN,SLD,-300,159.900000
C OESX DEC 19 3250,SLD,-16,131.900000


In [59]:
execs_STK = execs[ execs['contract_m_secType'] == 'STK' ]
execs_STK.head()

,contract_m_conId,contract_m_currency,contract_m_exchange,contract_m_expiry,contract_m_includeExpired,contract_m_localSymbol,contract_m_multiplier,contract_m_right,contract_m_secType,contract_m_strike,...,execution_m_exchange,execution_m_execId,execution_m_liquidation,execution_m_orderId,execution_m_orderRef,execution_m_permId,execution_m_price,execution_m_shares,execution_m_side,execution_m_time
7,13906061,JPY,TSEJ,None,False,9202,None,None,STK,0.0,...,TSEJ,00015232.5c81abe6.01.01,0,2147483647,None,2120345856,3994.00,-100,SLD,20190308 05:03:23
17,95548226,HKD,SEHK,None,False,6030,None,None,STK,0.0,...,SEHK,000116e7.5c820f3a.01.01,0,2147483647,None,2120345919,18.26,-2000,SLD,20190308 06:00:09
18,95548226,HKD,SEHK,None,False,6030,None,None,STK,0.0,...,SEHK,000116e7.5c820f51.01.01,0,2147483647,None,2120345919,18.22,-500,SLD,20190308 06:00:14
19,95548226,HKD,SEHK,None,False,6030,None,None,STK,0.0,...,SEHK,000116e7.5c820f57.01.01,0,2147483647,None,2120345919,18.22,-500,SLD,20190308 06:00:17
20,3884658,HKD,SEHK,None,False,1109,None,None,STK,0.0,...,SEHK,000116e7.5c820f96.01.01,0,2147483647,None,2120345861,30.25,-10000,SLD,20190308 06:01:15


In [60]:
execs_STK.groupby(["contract_m_localSymbol", "execution_m_side"]).agg( 
    {'execution_m_shares': ['sum'], 
     'execution_m_price': {'weighted_mean' :  
                           lambda x: np.average(x, weights=execs_STK.loc[x.index, "execution_m_shares"]) 
                          } 
    } 
)

c:\users\gregory\appdata\local\conda\conda\envs\notebook\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,,execution_m_shares,execution_m_price
,,sum,weighted_mean
contract_m_localSymbol,execution_m_side,,
1109,SLD,-10000,30.250000
291,SLD,-2000,30.050000
6030,SLD,-7500,18.337333
9202,SLD,-100,3994.000000
ADYEN,SLD,-10,634.400000
AMZN,BOT,60,1615.816667
BALN,SLD,-300,159.900000
CBK,SLD,-1600,6.709813


In [ ]:
res = requests.get(
    f'{HOST}/reports/ib/eod/v2', 
    json=json.loads(requests.get('http://127.0.0.1:443/executions').content),
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')))
json.loads(res.content)

# parse CSV report

In [ ]:
data = {}
with open('app/reports/data/U1160693_last.csv', 'r', encoding='utf-8-sig') as f:
    for row in f:
        for one_row in csv.reader([row]):
            break

        if one_row[0] != '':
            if one_row[0] in data:
                pass
            else:
                data[ one_row[0] ] = []

            data[ one_row[0] ].append(one_row)

data.keys()

In [ ]:
header = "Statement"
df_statement = pd.DataFrame(
                data=data[header][1:],
                columns=data[header][0])

df_statement = df_statement[ df_statement['Header'] == 'Data' ]
df_statement

In [ ]:
dateparser.parse("Juillet 27, 2018").date()

In [ ]:
df_statement[ df_statement['Nom champ'] == 'WhenGenerated' ]['Valeur champ'].values[0][:10]

In [ ]:
header = 'Informations instrument financier'

list_multiple_assetClasses = data[header]


dict_assetClasses = {}
k = 0
last_header = ''
for row in list_multiple_assetClasses:

    if row[1] == 'Header':
        last_header = str(k)
        k = k + 1
        dict_assetClasses[ last_header ] = []
    dict_assetClasses[ last_header ].append( row )


list_refData = []

cols = []

numeric_fields = ['Multiplicateur']

for assetClass in dict_assetClasses:
    cols = cols + dict_assetClasses[assetClass][0]
    cols = list(set(cols))

for assetClass in dict_assetClasses:
    df_refData = pd.DataFrame(
        data=dict_assetClasses[assetClass][1:],
        columns=dict_assetClasses[assetClass][0]
    )
    df_refData = df_refData[ df_refData['Header'] == 'Data' ]


    list_refData.append(df_refData)

#cols
df_refData = pd.concat(list_refData, axis=0, ignore_index=True, sort=False)

for field in numeric_fields:
    df_refData[field] = df_refData[field].replace(to_replace='--', value=np.nan)
    df_refData[field] = pd.to_numeric( df_refData[field].str.replace(',', '') )


date_fields = ['Expiration']
for field in date_fields:
    df_refData[field] = pd.to_datetime( df_refData[field], format="%Y-%m-%d" )


headers_refData = ['Catégorie d\'actifs', 'Symbole', 'Description', 'Conid', 'Multiplicateur', 'Expiration', 'Type', 'Prix de Levée' ]
headers_toShow = headers_refData


list_refData = df_refData.to_dict(orient='record')

refData = {}
for asset in list_refData:
    refData[ asset['Symbole'] ] = asset

df_refData = pd.DataFrame( list_refData )

df_refData

In [ ]:
def patch_missing_date(input_date):
    if pd.isnull(input_date):
        return None
    else:
        return input_date.strftime('%Y-%m-%d')

In [ ]:
df_refData.Expiration.apply(patch_missing_date)

In [ ]:
host = 'http://localhost:5000/'

In [ ]:
auth=HTTPBasicAuth(os.environ.get('BASIC_AUTH_USERNAME'), os.environ.get('BASIC_AUTH_PASSWORD') )

In [ ]:
requests.get(host, auth=auth )

In [ ]:
res = requests.get(host + 'reports/ib/eod/fxrate', auth=auth)
df_fxRate = pd.DataFrame( [{'Devise': 'EUR', 'Taux': 1.0}] + json.loads(res.content) )
df_fxRate.set_index('Devise', inplace=True)
df_fxRate

In [ ]:
df_fxRate.loc['EUR']['Taux']

In [ ]:
res = requests.get(host + 'reports/ib/eod/transactions', auth=auth)
df = pd.DataFrame( json.loads(res.content) )
df.head()

In [ ]:
res = requests.get('http://localhost:443/executions')
executions = json.loads(res.content)
executions

In [ ]:
res = requests.get(host + 'reports/ib/eod', auth=auth, json={'execDetails': []})
res.content

In [ ]:
res = requests.get(host + 'reports/ib/eod', auth=auth, json={'execDetails': []})
df_pos = pd.DataFrame( json.loads(res.content) )
df_pos.head()

In [ ]:
res = requests.get(host + 'reports/ib/eod', auth=auth, json=executions )
df_pos = pd.DataFrame( json.loads(res.content) )

df_pos

In [ ]:
data = {
    'execId': 'exec' + str( random.randint(1000, 10000000) ),
    'orderId': str( random.randint(1000, 10000000) ),
    'time': '20180604  13:32:52',
    'acctNumber': 'U' + str( random.randint(1000, 10000000) ),
    'exchange': 'CBOE',
    'side': 'BOT',
    'shares': 13,
    'cumQty': -13,
    'price': 7.23,
    'avgPrice': -7.23,
    'permId': str( random.randint(1000, 10000000) ),
    
    'localSymbol': 'GOOGL 6 C1200.25 Equity',
    'secType': 'OPT',
    'symbol': 'GOOGL',
    'currency': 'USD',
    'primaryExchange': '',
    'lastTradeDateOrContractMonth': '20180615',
    'multiplier': 100,
    'strike': 1200.25,
    'right': 'C'
    
}
data

In [ ]:
data2 = {
    'execId': 'exec' + str( random.randint(1000, 10000000) ),
    'orderId': str( random.randint(1000, 10000000) ),
    'time': '20180604  13:32:52',
    'acctNumber': 'U' + str( random.randint(1000, 10000000) ),
    'exchange': 'CBOE',
    'side': 'BOT',
    'shares': 13,
    'cumQty': -13,
    'price': 7.23,
    'avgPrice': -7.23,
    'permId': str( random.randint(1000, 10000000) ),
    
    'localSymbol': 'GOOGL 6 C' + str( random.randint(900, 1400) ) + '.25 Equity',
    'secType': 'OPT',
    'symbol': 'GOOGL',
    'currency': 'USD',
    'primaryExchange': '',
    'lastTradeDateOrContractMonth': '20180615',
    'multiplier': 100,
    'strike': 1200.25,
    'right': 'C'
    
}

res = requests.post(host + 'executions', auth=auth,json=data2 )
#res.content
json.loads(res.content)

In [ ]:
res = requests.get(host + 'bridge/missing', auth=auth )
#res.content
missings = json.loads(res.content)
missings['ib']

In [ ]:
for ib in missings['ib']:
    data3 = {
        "localSymbol": ib,
        "ticker": ib.upper()
    }
    #res = requests.post(host + 'bridge', auth=auth,json=data3 )
    #print(res.content)

In [ ]:
data3 = {
    'ticker': 'GOOGL US EQUITY'
}

res = requests.post(host + 'bridge/bbg/ib', auth=auth,json=data3 )
#res.content
json.loads(res.content)

In [ ]:
data4 = {
    'tickers': ['GOOGL US EQUITY', 'AAPL US equity', 'blub uq equity']
}

res = requests.post(
    f'{HOST}/bridge/bbgs/ibs', 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')), 
    json=data4 )
#res.content
json.loads(res.content)

In [ ]:
res = requests.get(host + 'bridge/backup', auth=auth )
#res.content
json.loads(res.content)

In [ ]:
data5 = {
    'tickers': ['GOOGL US EQUITY', 'AAPL US equity', 'blub uq equity']
}


res = requests.post(
    f'{HOST}/ibsymbology/bbgs/ibs', 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD')), 
    json=data5 )
#res.content
json.loads(res.content)

In [ ]:
res = requests.get(
    f'{HOST}/ibsymbology/recent/2018-11-19 16:17:00', 
    auth=HTTPBasicAuth(os.getenv('BASIC_AUTH_USERNAME'), os.getenv('BASIC_AUTH_PASSWORD'))
)
#res.content
json.loads(res.content)